In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from local.torch_basics import *
from local.test import *
from local.basics import *
from local.data.all import *
from local.vision.core import *
from local.notebook.showdoc import show_doc
from local.audio.core import *
from local.audio.augment import *
from local.vision.learner import *
from local.vision.models.xresnet import *
from local.metrics import *
from local.callback.schedule import *
import torchaudio
from fastprogress import progress_bar as pb
from numba import njit, prange
from rocket import ROCKET, ROCKETMAX

In [ ]:
PATH_PROCESSED =  PATH_ORIGINAL/"250-speakers/.processed"
x = AudioGetter("", recurse=True, folders=None)
files_250  = x(PATH_PROCESSED)
oa = OpenAudio(files_250)

In [ ]:
files_250

(#44655) [/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/_KymcHdEW0U/00034.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/2HgpwyiMUEE/00002.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/11xxoaj4aEA/00001.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/gC8wHtwhnZw/00036.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/gC8wHtwhnZw/00038.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/gC8wHtwhnZw/00037.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/DwK2JNH10zE/00013.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/DwK2JNH10zE/00012.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/DwK2JNH10zE/00011.wav,/home/jupyter/.fastai/data/250_speakers/250-speakers/.processed/id09028/4f8IoTgW8z0/00005.wav...]

In [ ]:
def AudioBlock(cls=AudioItem): return TransformBlock(type_tfms=cls.create, batch_tfms=IntToFloatTensor)

In [ ]:
def extract_data(dbunch):
    train_dl, valid_dl = dbunch
    sigs_train, labels_train = [],[]
    for batch in pb(train_dl):
        sigs_train.append(batch[0])
        labels_train.append(batch[1])
    sigs_valid, labels_valid = [],[]
    for batch in pb(valid_dl):
        sigs_valid.append(batch[0])
        labels_valid.append(batch[1])
    sigs_train = torch.cat(sigs_train, axis=0).cpu().numpy()
    labels_train = torch.cat(labels_train, axis=0).cpu().numpy()
    sigs_valid = torch.cat(sigs_valid, axis=0).cpu().numpy()
    labels_valid = torch.cat(labels_valid, axis=0).cpu().numpy()
    return sigs_train, labels_train, sigs_valid, labels_valid

In [ ]:
auds = DataBlock(blocks=(AudioBlock, CategoryBlock),  
                 get_items=get_audio_files, 
                 splitter=RandomSplitter(),
                 get_y=lambda x: str(x).split('/')[-3][3:])

In [ ]:
def Normalize(m, s):
    def _inner(ai:AudioItem)->AudioItem:
        sig = ai.sig.clone()
        norm_sig = (sig - m)/(s + 1e-8)
        return AudioItem((norm_sig, ai.sr, ai.path))
    return _inner

In [ ]:
norm = Normalize(1.5217e-05, 0.0571)

In [ ]:
total_mean = 0
total_std = 0
total_mean2 = 0
total_std2 = 0
num_files=len(files_250[0:1000])
for i in pb(range(num_files)):
    total_mean += oa(i).sig.mean()
    total_std  += oa(i).sig.std()
    total_mean2 += norm(oa(i)).sig.mean()
    total_std2  += norm(oa(i)).sig.std()
mn, std = total_mean/num_files, total_std/num_files
norm_mn, norm_std = total_mean2/num_files, total_std2/num_files

In [ ]:
mn

tensor(1.5217e-05)

In [ ]:
std

tensor(0.0571)

In [ ]:
norm_mn, norm_std

(tensor(-6.4499e-09), tensor(0.9999))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rocket = ROCKET(1, 32000, n_kernels=100, kss=[7, 9, 11], stride=5).to(device)

In [ ]:
def GetSig():
    def _inner(ai:AudioItem)->AudioItem:
        return ai.sig
    return _inner

In [ ]:
crop_2000ms = CropSignal(2000)
tfms = [crop_2000ms, norm, GetSig(),]
gpu_tfms = [Cuda(), rocket]
dbunch = auds.databunch(PATH_PROCESSED, item_tfms=tfms,  batch_tfms=gpu_tfms, bs=96)

In [ ]:
%%time
x_train, y_train, x_valid, y_valid = extract_data(dbunch)

CPU times: user 29.6 s, sys: 8.68 s, total: 38.3 s
Wall time: 39.9 s


In [ ]:
x_train, x_valid = map(lambda x: x.squeeze(1), (x_train, x_valid))

In [ ]:
x_train.shape

(35712, 200)

In [ ]:
%%time
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
classifier.fit(x_train, y_train)
classifier.score(x_valid, y_valid)

CPU times: user 8.33 s, sys: 1.49 s, total: 9.82 s
Wall time: 2.66 s


0.21430970775948943

In [ ]:
rocketmax = ROCKETMAX(1, 32000, n_kernels=100, kss=[7, 9, 11], stride=5).to(device)

In [ ]:
gpu_tfms_max = [Cuda(), rocketmax]
dbunch = auds.databunch(PATH_PROCESSED, item_tfms=tfms,  batch_tfms=gpu_tfms_max, bs=96)

In [ ]:
%%time
x_train, y_train, x_valid, y_valid = extract_data(dbunch)

CPU times: user 1min 13s, sys: 28.3 s, total: 1min 41s
Wall time: 1min 42s


In [ ]:
x_train.shape

(35712, 1, 1400)

In [ ]:
x_train, x_valid = map(lambda x: x.squeeze(1), (x_train, x_valid))

In [ ]:
%%time
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
classifier.fit(x_train, y_train)
classifier.score(x_valid, y_valid)

CPU times: user 28.3 s, sys: 4.23 s, total: 32.5 s
Wall time: 10.2 s


0.34934497816593885